In [ ]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal

from cropmask.misc import parse_yaml, make_dirs

In [ ]:
from cropmask.preprocess import PreprocessWorkflow
wflow = PreprocessWorkflow("/az-ml-container/configs/preprocess_config_pytest.yaml", 
                                 "/az-ml-container/western_nebraska_landsat_scenes_pytest/LT050320312005011601T1-SC20190418222311/",
                                 "/az-ml-container/external_pytest/nebraska-center-pivots-2005/nbextent-clipped-to-western.geojson")

In [ ]:
directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

In [ ]:
shp_frame = gpd.read_file(wflow.source_label_path)

In [ ]:
# keeps the class of interest if it is there and the polygon of raster extent
meta = wflow.meta.copy()
meta.update({'count':1})
tifname = os.path.splitext(os.path.basename(wflow.source_label_path))[0] + ".tif"
wflow.rasterized_label_path = os.path.join(wflow.NEG_BUFFERED, tifname)
with rasterio.open(wflow.rasterized_label_path, "w+", **wflow.meta) as out:
    out_arr = out.read(1)
    shp_frame = shp_frame.loc[shp_frame.area > wflow.small_area_filter]
    shp_frame["geometry"] = shp_frame["geometry"].buffer(wflow.neg_buffer)
    shp_frame = shp_frame.loc[shp_frame.geometry.is_empty==False]
    # https://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python#151861
    shapes = (
        (geom.__geo_interface__, value)
        for geom, value in zip(shp_frame.geometry, shp_frame.ID)
    )

    burned = features.rasterize(
        shapes=shapes,
        fill=0,
        out_shape=(meta['height'],meta['width']),
        transform=out.transform,
        default_value=1,
    )
    burned[burned < 0] = 0
    burned[burned > 0] = 1
    burned = burned.astype(np.int16, copy=False)
    out.write(burned, 1)

In [ ]:
from rasterio.features import is_valid_geom

In [ ]:
shp_frame['geometry'].iloc[3973].is_empty

In [ ]:
    try: 
        img_chip_list = os.listdir(wflow.GRIDDED_IMGS)
        label_chip_list = os.listdir(wflow.GRIDDED_LABELS)
        assert img_chip_list # make sure that this is not empty
        assert label_chip_list
        remove_dirs(directory_list)
    except AssertionError:
        remove_dirs(directory_list)
        print("The product list is empty, check this path: "+ wflow.GRIDDED_IMGS)

In [ ]:
wflow.negative_buffer_and_small_filter(-31, 100)